In [ ]:
import cv2
import torch
torch.cuda.empty_cache()
from PIL import Image
from utils.general import xyxy2xywh
import numpy as np
import os
import time

# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model='last.pt')  # custom model
model.conf = 0.4

In [ ]:
import glob
import numpy

dirr = 'E:/images/left_view'    ############################### USER INPUT

save_img_dir = r'C:\Users\CIEDEV\Desktop\Nabin\saved'  #########################     USERINPUT

left_files = glob.glob(dirr + '/**/*.tiff', recursive=True)[:100]  #--> ### UserInput

img_name = [ os.path.basename(img) for img in left_files]

In [ ]:
result = model(left_files)  # applying model to img
detection = result.xyxy  # coordinates of 2/3 bottles each image 
i = 0 
dw = 1920 # img width
dh = 1200 # img height
final_coords = [] #coords for the middle image
for det in detection:

    coords = det[:,:4].cpu().numpy()
    x,y = dw/2, dh/2
    
    if len(coords) == 1:     ## code to identify center image and append to the final coord
         final_coords.append(coords)
    elif len(detection) == 2:
        c1 = coords[0]
        c2 = coords[1]
        c1_x,c1_y = (c1[0] + c1[2])/2, (c1[1] + c1[3])/2
        c2_x,c2_y = (c2[0] + c2[2])/2, (c2[1] + c2[3])/2
        dist1 =  abs(c1_x - x) + abs(c1_y - y)
        dist2 =  abs(c2_x - x) + abs(c2_y - y)
        if dist1 < dist2:
            final_coords.append(c1)
        else:
            final_coords.append(c2)
    elif len(coords) == 3:
        c1 = coords[0]
        c2 = coords[1]
        c3 = coords[2]
        c1_x,c1_y = (c1[0] + c1[2])/2, (c1[1] + c1[3])/2
        c2_x,c2_y = (c2[0] + c2[2])/2, (c2[1] + c2[3])/2
        c3_x,c3_y = (c3[0] + c3[2])/2, (c3[1] + c3[3])/2
        dist1 =  abs(c1_x - x) + abs(c1_y - y)
        dist2 =  abs(c2_x - x) + abs(c2_y - y)
        dist3 =  abs(c3_x - x) + abs(c3_y - y)
        if dist1 < dist2 and dist1 < dist3:
            final_coords.append(c1)
        elif dist2 < dist1 and dist2 < dist3 :
            final_coords.append(c2)
        else :
            final_coords.append(c3)
    

for index, coord in enumerate(final_coords):  
    img = left_files[index] 
    
    name_img = img_name[index]
#     final_coords = np.float32(final_coords)
#     cv2.rectangle(img,(coord[0],coord[1]),(coord[2],coord[3]), (0,255,0),3)  ## draw rect for coords of center image
    try:
        crop_img = img[int(coord[1]-10):int(coord[3]), int(coord[0]): int(coord[2])]
    except Exception:
        pass
#     cv2.imshow(str(img),crop_img)
#     cv2.waitKey(0)
    os.chdir(save_img_dir)  # Save image to directory mentioned
    cv2.imwrite(name_img, crop_img)
    print('Done')

In [ ]:
# coords = detection[:,:4].cpu().numpy()

# x,y = dw/2, dh/2
# if len(coords) == 1:
#     final_coords = coords
# elif len(detection) == 2:
#     c1 = coords[0]
#     c2 = coords[1]
#     c1_x,c1_y = (c1[0] + c1[2])/2, (c1[1] + c1[3])/2
#     c2_x,c2_y = (c2[0] + c2[2])/2, (c2[1] + c2[3])/2
#     dist1 =  abs(c1_x - x) + abs(c1_y - y)
#     dist2 =  abs(c2_x - x) + abs(c2_y - y)
#     if dist1 < dist2:
#         final_coords = c1
#     else:
#         final_coords = c2
# elif len(coords) == 3:
#     c1 = coords[0]
#     c2 = coords[1]
#     c3 = coords[2]
#     c1_x,c1_y = int(c1[0] + c1[2])/2, (c1[1] + c1[3])/2
#     c2_x,c2_y = int(c2[0] + c2[2])/2, (c2[1] + c2[3])/2
#     c3_x,c3_y = int(c3[0] + c3[2])/2, (c3[1] + c3[3])/2
#     dist1 =  abs(c1_x - x) + abs(c1_y - y)
#     dist2 =  abs(c2_x - x) + abs(c2_y - y)
#     dist3 =  abs(c3_x - x) + abs(c3_y - y)
#     if dist1 < dist2 and dist1 < dist3:
#         final_coords = c1
#     elif dist2 < dist1 < dist3 :
#         final_coords = c2
#     else :
#         final_coords = c3
# print(final_coords)
# # final_coords = np.float32(final_coords)

# cv2.rectangle(img,(final_coords[0],final_coords[1]),(final_coords[2],final_coords[3]), (0,255,0),3)
# cv2.imshow('Img',img)
# cv2.waitKey(0)